 Import libraries

In [1]:
!pip install tensorflow_addons
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
%tensorflow_version 2.x
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Sequential
!pip3 install pillow-heif
#this is for importing HEIC images with Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 7.3 MB/s 
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.5 MB 6.6 MB/s 


Mount Notebook onto drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Create appropriate directory for images to be processed

In [3]:
from pathlib import Path
# assign directory
directory = "/content/drive/MyDrive/Flower Images/Diablo Cosmo"
 
# iterate over files in
# that directory
files = Path(directory).glob('*')
#for file in files:
    #print(file)

Bruna's greyscale function from our teams Gogs that we created earlier in the semester. Did not end up using as takes too long.

In [ ]:
from PIL import Image, ImageFilter, ImageEnhance
from pillow_heif import register_heif_opener

def both_filters_greyscale1(image):

    #kernel to traverse x
    horizontal_filter = [[-1,0,1],
                        [-2,0,2],
                        [-1,0,1]]

    #kernel to traverse y
    vertical_filter =   [[-1,-2,-1],
                        [0,0,0],
                        [1,2,1]]

    height, width = image.shape
    
    new_image = np.zeros_like(image)
    
    for i in range(1, height - 2):
        for j in range(1, width-2):
            local_pixels = image[i-1:i+2, j-1:j+2]
            
            horizontal_transformed_pixels = horizontal_filter * local_pixels
            horizontal_score = (horizontal_transformed_pixels.sum())    #these values are not plot to 0-1 because otherwise 
                                                                        #the result becomes too dim.
        
            vertical_transformed_pixels = vertical_filter * local_pixels 
            vertical_score = (vertical_transformed_pixels.sum())
            
            edge_score = (vertical_score**2 + horizontal_score**2)**0.5
            new_image[i][j] = edge_score 
    
    return new_image

Preprocessing program, based off of Bruna's original preprocessing Colab notebook.

In [4]:
from PIL import Image, ImageFilter, ImageEnhance
from pillow_heif import register_heif_opener


register_heif_opener()

name = 0; #I just used numbers for named, but we can choose some other naming convention.
for flower in files:
  name = name + 1
  image = Image.open(flower)#("/content/drive/MyDrive/Flower Images/Vanilla Ice Sunflower/Copy of IMG_9713.HEIC");
  
  #Convert Image to greyscale
  image = image.convert("L")
  # np_image = both_filters_greyscale1(np.asarray(image))
  # image = Image.fromarray(np.uint8(np_image)).convert('L')
  # image.save("/content/drive/MyDrive/preprocessing/greyscale/"+ str(53) + ".jpg")

  # Blur image with Gaussian blur
  image = image.filter(ImageFilter.GaussianBlur(radius = 3))
  # image.save("/content/drive/MyDrive/preprocessing/gb2/"+ str(53) + ".jpg")
   

  # Perform edge detection with Sobel Edge detection using cv2
  # Considered using Pillow's FIND_EDGES, but learned that it implements Laplacian Edge detection, when we were intending to use Sobel.
  x = cv2.convertScaleAbs(cv2.Sobel(np.asarray(image), cv2.CV_64F, 1, 0, ksize=3))
  y = cv2.convertScaleAbs(cv2.Sobel(np.asarray(image), cv2.CV_64F, 0, 1, ksize=3))
  image = Image.fromarray(np.uint8(cv2.addWeighted(x, 0.5, y, 0.5, 0))).convert('L')
  # image.save("/content/drive/MyDrive/preprocessing/edge/"+ str(53) + ".jpg")


  # Increasing brightness:
  image = image.point(lambda i: i * 4)
  # image.save("/content/drive/MyDrive/preprocessing/enhance/"+ str(53) + ".jpg")

  # Perform thresholding
  image = image.point( lambda p: 255 if p > 100 else 0 )

  #Save final preprocessed image
  image.save("/content/drive/MyDrive/Preprocessing/Diablo Cosmo/"+ str(name) + ".jpg")


KeyboardInterrupt: ignored